### Example extraction of a paper from the Royal Society of Chemistry

#### Import the required modules

In [ ]:
from tadf_models.models import *
from chemdataextractor.doc import Document, Citation
import json
import importlib.resources
from chemdataextractor.model import ThemeCompound
import os
from pprint import pprint

#### Load the compound blocklists

In [ ]:
tadf_blocklist = importlib.resources.files('tadf_models').joinpath('tadf_blocklist_6_more_abbrev_enriched').read_text(encoding='utf8')

element_blocklist = importlib.resources.files('tadf_models').joinpath('elements').read_text(encoding='utf8')
tadf_blocklist = tadf_blocklist.split('\n')
element_blocklist = element_blocklist.split('\n')
ThemeCompound.name_blocklist = ThemeCompound.name_blocklist + tadf_blocklist + element_blocklist

In [ ]:
class TADFExtractor():
    """Class that extracts a document and saves records in JSON format."""

    def __init__(self, paper_root, models):
        """
        Initalise the class with the directory and name of the document, the models to extract, and the save directory.
        :param str paper_root: directory that stores the paper.
        :param str save_root: directory that the JSON results are written into.
        :param str filename: The name of output file name.
        :param list[chemdataextractor.models.BaseModel] models: list of ChemDataExtractor models to extract.
        """
        self.dic = None
        self.paper_root = paper_root
        self.models = models
        self.count = 0

    def is_incomplete_paper(self, fstring):
        """
        Method that check if a document is complete.
        :param str fstring: document in plain text string.
        :returns: if the document is complete or not.
        :rtype: Boolean.
        """
        print('Checking completeness.')
        if "(Note: The full text of this document is currently only available in the" in fstring:
            # RSC case
            return True
        elif "<xocs:rawtext" in fstring:
            # Elsevier case
            return True
        return False

    def load_document(self, file):
        """
        Method that load a document into a chemdataextractor.doc.Document object.
        :param str file: path to the document.
        :returns: the Document object and if the document is incomplete.
        :rtype: chemdataextractor.doc.Document, Boolean
        """

        d = Document.from_file(file)
        incomplete = False
        with open(file, encoding="utf8") as f:
            fstring = f.read()
        if self.is_incomplete_paper(fstring):
            incomplete = True

        return d, incomplete

    def extraction(self, file):
        """
        Method that load a extract records from a document and save them in JSON.
        :param str file: path to the document.
        """

        print(f'Attempting to extract {file}.')

        if True:
            doc, incomplete = self.load_document(os.path.join(self.paper_root, file))

            if incomplete:
                print(f'{file} is incomplete!')
                return
            
            else:
                print(f"{file} is complete, extracting.")

                try:
                    doi = doc.metadata.doi

                except IndexError:
                    print("MetaData is Empty.")
                    doi = file.name

                doc.models = self.models
                doc.skip_elements = [Citation]
                rough = doc.records

                records = []
                for r in rough:
                    self.dic = r.serialize()
                    self.dic['doi'] = doi
                    self.dic['record_method'] = r.record_method
                    records.append(self.dic)
                    self.count += 1

                print(f"{self.count} records in total")
                print(f"{file} extracted.")
                return records

#### Start the extraction!

In [ ]:
# The extraction will takes ~5 mins to run.
mdb = TADFExtractor(paper_root="papers_for_extraction", 
                    models=[STSplit])
records = mdb.extraction("test.html")

In [ ]:
### print the extracted records ###
pprint(records)